In [23]:
import sys
import os
sys.path.append("/home/nakamura/network_dissection/NetDissect/src")
os.chdir("/home/nakamura/network_dissection/NetDissect/")

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import pickle

from viewprobe import NetworkProbe
from category_info import category_info
script = "/home/nakamura/network_dissection/NetDissect/research/top_iou_detected_ratio_1227.ipynb"
% matplotlib inline

In [24]:
# define models
alex = {}
alex["name"] = "alexnet_imagenet"
alex["directory"] = "./dissection/alexnet_imagenet_full_conv_384/"
alex["blobs"] = ["conv1", "conv2", "conv3", "conv4", "conv5", "fc6-conv", "fc7-conv", "fc8-conv"]

vgg_material = {}
vgg_material["name"] = "vgg_material"
vgg_material["directory"] = "./dissection/vgg_material_full_conv/"
vgg_material["blobs"] = ["conv1_2", "conv2_2","conv3_3", "conv4_3", "conv5_3", "fc6-conv", "fc7-conv", "fc8-conv"]

sentibank = {}
sentibank["name"] = "sentibank"
sentibank["directory"] = "./dissection/caffe_sentibank_full_conv/"
sentibank["blobs"] = ["conv1", "conv2","conv3", "conv4", "conv5", "fc6-conv", "fc7-conv", "fc8-conv"]

vgg_face = {}
vgg_face["name"] = "vgg_face"
vgg_face["directory"] = "./dissection/VGG_FACE_full_conv/"
vgg_face["blobs"] = ["conv1_2", "conv2_2","conv3_3", "conv4_3", "conv5_3", "fc6-conv", "fc7-conv", "fc8-conv"]

models = [alex, vgg_material, sentibank, vgg_face]

In [25]:
def detect_concept_max(model, blob):
    
    # read iou data
    path= os.path.join(model["directory"], blob + "-iou.csv")
    save_file = os.path.join("./research/concept_detection/", model["name"] + "_" + blob + "_threshold.pkl")
    iou = pd.read_csv(path, index_col=0)
    
    argmax_iou = iou.idxmax(axis=1).tolist()

    detected_set = set(argmax_iou)
    
    # save
    #with open(save_file, "wb") as f:
     #   pickle.dump(detected_set, f)
        
    return detected_set

In [26]:
def detect_concept_thres_max(model, blob, threshold=0.05):
    max_detected = detect_concept_max(model, blob)
    thresh_detected = detect_concept_threshold(model, blob, threshold)
    return max_detected & thresh_detected

In [27]:
def detect_concept_threshold(model, blob, threshold=0.05):
     
    path= os.path.join(model["directory"], blob + "-iou.csv")
    save_file = os.path.join("./research/concept_detection/", model["name"] + "_" + blob + "_threshold.pkl")
    
    iou = pd.read_csv(path, index_col=0)
    
    is_thres = iou > threshold

    # is_detected = isthres.any(axis=0)
    num_detec = is_thres.sum(axis=0)
    is_detec = num_detec[num_detec > 0]

    
    detected_set = set(is_detec.index.tolist())
    
    
    # save
    with open(save_file, "wb") as f:
        pickle.dump(detected_set, f)
        
    return detected_set

## Comparison between Alexnet and VGG-material (threshold=0.05)

In [28]:
# comparison between alexnet and vgg material
for blob in range(8):
    alex_ = detect_concept_threshold(alex, alex["blobs"][blob])
    vgg_ = detect_concept_threshold(vgg_material ,vgg_material["blobs"][blob])
    print("alexnet:{}, vgg material:{}\n".format(alex["blobs"][blob], vgg_material["blobs"][blob]))
    print("unique for alex:\n\n{}\n".format(", ".join(list(alex_ - vgg_))))
    print("unique for vgg:\n\n{}\n".format(", ".join(list(vgg_ - alex_))))
    print("common concept:\n\n{}\n".format(", ".join(list(vgg_ & alex_))))

alexnet:conv1, vgg material:conv1_2

unique for alex:

bubbly, veined, frilly, dotted, blue-c, red-c, sky, yellow-c, purple-c, grass, polka-dotted

unique for vgg:



common concept:



alexnet:conv2, vgg material:conv2_2

unique for alex:

flecked, meshed, banded, knitted, zigzagged, highway-s, sky, yellow-c, purple-c, lined, frilly, grid, crosshatched, striped, green-c, veined, pink-c, blue-c, porous, orange-c, perforated, grass, woven

unique for vgg:



common concept:

red-c, lacelike

alexnet:conv3, vgg material:conv3_3

unique for alex:

meshed, swirly, grooved, skyscraper-s, muzzle, dotted, sky, skyscraper, waffled, studded, polka-dotted, wheel, plant, ball, frilly, food, fibrous, ball_pit-s, striped, cracked, mountain_snowy-s, chequered, veined, blue-c, matted, tree, dog, cat, person, honeycombed, grass, head

unique for vgg:

hand

common concept:

zigzagged, cobwebbed, red-c, banded, perforated, grid, lacelike, crosshatched, lined, woven

alexnet:conv4, vgg material:conv4_3


## Comparison between Alexnet and VGG-FACE (threshold=0.05)

In [31]:
# comparison between alexnet and vgg material
for blob in range(8):
    alex_ = detect_concept_threshold(alex, alex["blobs"][blob])
    vgg_ = detect_concept_threshold(vgg_face ,vgg_face["blobs"][blob])
    print("alexnet:{}, vgg face:{}\n".format(alex["blobs"][blob], vgg_face["blobs"][blob]))
    print("unique for alex:\n\n{}\n".format(", ".join(list(alex_ - vgg_))))
    print("unique for vgg face:\n\n{}\n".format(", ".join(list(vgg_ - alex_))))
    print("common concept:\n\n{}\n".format(", ".join(list(vgg_ & alex_))))

alexnet:conv1, vgg face:conv1_2

unique for alex:

bubbly, veined, frilly, dotted, blue-c, red-c, sky, yellow-c, purple-c, grass, polka-dotted

unique for vgg face:



common concept:



alexnet:conv2, vgg face:conv2_2

unique for alex:

flecked, meshed, porous, knitted, zigzagged, highway-s, sky, yellow-c, purple-c, lacelike, lined, frilly, grid, crosshatched, striped, green-c, veined, pink-c, banded, orange-c, perforated, grass, woven

unique for vgg face:

white-c

common concept:

red-c, blue-c

alexnet:conv3, vgg face:conv3_3

unique for alex:

meshed, swirly, tree, grooved, skyscraper-s, muzzle, zigzagged, dotted, sky, skyscraper, lacelike, studded, polka-dotted, wheel, plant, ball, frilly, food, fibrous, waffled, grid, ball_pit-s, crosshatched, striped, cracked, mountain_snowy-s, chequered, veined, blue-c, cobwebbed, matted, red-c, woven, grass, banded, dog, cat, person, perforated, honeycombed, lined, head

unique for vgg face:



common concept:



alexnet:conv4, vgg face:conv

## Other Comparisons

In [18]:
# comparison between alexnet and vgg material
for blob in range(8):
    alex_ = detect_concept_threshold(alex, alex["blobs"][blob])
    vgg_ = detect_concept_threshold(vgg_material ,vgg_material["blobs"][blob])
    print("alexnet:{}, vgg material:{}\n".format(alex["blobs"][blob], vgg_material["blobs"][blob]))
    print("unique for alex:\n\n{}\n".format(", ".join(list(alex_ - vgg_))))
    print("unique for vgg:\n\n{}\n".format(", ".join(list(vgg_ - alex_))))
    print("common concept:\n\n{}\n".format(", ".join(list(vgg_ & alex_))))

alexnet:conv1, vgg material:conv1_2

unique for alex:

bubbly, veined, frilly, dotted, blue-c, red-c, sky, yellow-c, purple-c, grass, polka-dotted

unique for vgg:



common concept:



alexnet:conv2, vgg material:conv2_2

unique for alex:

flecked, meshed, banded, knitted, zigzagged, highway-s, sky, yellow-c, purple-c, lined, frilly, grid, crosshatched, striped, green-c, veined, pink-c, blue-c, porous, orange-c, perforated, grass, woven

unique for vgg:



common concept:

red-c, lacelike

alexnet:conv3, vgg material:conv3_3

unique for alex:

meshed, swirly, grooved, skyscraper-s, muzzle, dotted, sky, skyscraper, waffled, studded, polka-dotted, wheel, plant, ball, frilly, food, fibrous, ball_pit-s, striped, cracked, mountain_snowy-s, chequered, veined, blue-c, matted, tree, dog, cat, person, honeycombed, grass, head

unique for vgg:

hand

common concept:

zigzagged, cobwebbed, red-c, banded, perforated, grid, lacelike, crosshatched, lined, woven

alexnet:conv4, vgg material:conv4_3


In [ ]:
# comparison between alexnet and vgg material
for blob in range(8):
    alex_ = detect_concept_thres_max(alex, alex["blobs"][blob])
    vgg_ = detect_concept_thres_max(vgg_material ,vgg_material["blobs"][blob])
    print("alexnet:{}, vgg material:{}\n".format(alex["blobs"][blob], vgg_material["blobs"][blob]))
    print("unique for alex:\n{}\n".format(list(alex_ - vgg_)))
    print("unique for vgg:\n{}\n".format(list(vgg_ - alex_)))
    print("common concept:\n{}\n".format(list(vgg_ & alex_)))

In [13]:
# comparison between vgg face and vgg material
for blob in range(8):
    face = detect_concept_threshold(vgg_face, vgg_face["blobs"][blob])
    vgg_ = detect_concept_threshold(vgg_material ,vgg_material["blobs"][blob])
    print("vgg_face:{}, vgg material:{}\n".format(vgg_face["blobs"][blob], vgg_material["blobs"][blob]))
    print("unique for vgg face:\n{}\n".format(list(face - vgg_)))
    print("unique for vgg material:\n{}\n".format(list(vgg_ - face)))
    print("common concept:\n{}\n".format(list(vgg_ & face)))

vgg_face:conv1_2, vgg material:conv1_2

unique for vgg face:
[]

unique for vgg material:
[]

common concept:
[]

vgg_face:conv2_2, vgg material:conv2_2

unique for vgg face:
['white-c', 'blue-c']

unique for vgg material:
['lacelike']

common concept:
['red-c']

vgg_face:conv3_3, vgg material:conv3_3

unique for vgg face:
[]

unique for vgg material:
['zigzagged', 'cobwebbed', 'red-c', 'banded', 'hand', 'perforated', 'grid', 'lacelike', 'crosshatched', 'lined', 'woven']

common concept:
[]

vgg_face:conv4_3, vgg material:conv4_3

unique for vgg face:
[]

unique for vgg material:
['meshed', 'swirly', 'porous', 'head', 'hair', 'bubbly', 'zigzagged', 'dotted', 'sprinkled', 'pitted', 'studded', 'polka-dotted', 'scaly', 'wheel', 'plant', 'ball', 'fibrous', 'hand', 'lacelike', 'ball_pit-s', 'striped', 'cracked', 'chequered', 'veined', 'cobwebbed', 'matted', 'paisley', 'grass', 'tree', 'dog', 'cat', 'person', 'waffled', 'perforated', 'honeycombed', 'lined', 'woven']

common concept:
['banded

In [10]:
# comparison between vgg face and vgg material
for blob in range(8):
    face = detect_concept_threshold(vgg_face, vgg_face["blobs"][blob])
    vgg_ = detect_concept_threshold(vgg_material ,vgg_material["blobs"][blob])
    print("vgg_face:{}, vgg material:{}\n".format(vgg_face["blobs"][blob], vgg_material["blobs"][blob]))
    print("unique for vgg face:\n{}\n".format(list(face - vgg_)))
    print("unique for vgg:\n{}\n".format(list(vgg_ - face)))
    print("common concept:\n{}\n".format(list(vgg_ & face)))

vgg_face:conv1_2, vgg material:conv1_2

unique for vgg face:
[]

unique for vgg:
[]

common concept:
[]

vgg_face:conv2_2, vgg material:conv2_2

unique for vgg face:
['white-c', 'blue-c']

unique for vgg:
['lacelike']

common concept:
['red-c']

vgg_face:conv3_3, vgg material:conv3_3

unique for vgg face:
[]

unique for vgg:
['zigzagged', 'cobwebbed', 'red-c', 'banded', 'hand', 'perforated', 'grid', 'lacelike', 'crosshatched', 'lined', 'woven']

common concept:
[]

vgg_face:conv4_3, vgg material:conv4_3

unique for vgg face:
[]

unique for vgg:
['meshed', 'swirly', 'porous', 'head', 'hair', 'bubbly', 'zigzagged', 'dotted', 'sprinkled', 'pitted', 'studded', 'polka-dotted', 'scaly', 'wheel', 'plant', 'ball', 'fibrous', 'hand', 'lacelike', 'ball_pit-s', 'striped', 'cracked', 'chequered', 'veined', 'cobwebbed', 'matted', 'paisley', 'grass', 'tree', 'dog', 'cat', 'person', 'waffled', 'perforated', 'honeycombed', 'lined', 'woven']

common concept:
['banded', 'grid']

vgg_face:conv5_3, vgg ma

In [11]:
# comparison between vgg face and vgg material
for blob in range(8):
    face = detect_concept_threshold(vgg_face, vgg_face["blobs"][blob])
    vgg_ = detect_concept_threshold(vgg_material ,vgg_material["blobs"][blob])
    print("vgg_face:{}, vgg material:{}\n".format(vgg_face["blobs"][blob], vgg_material["blobs"][blob]))
    print("unique for vgg face:\n{}\n".format(list(face - vgg_)))
    print("unique for vgg:\n{}\n".format(list(vgg_ - face)))
    print("common concept:\n{}\n".format(list(vgg_ & face)))

vgg_face:conv1_2, vgg material:conv1_2

unique for vgg face:
[]

unique for vgg:
[]

common concept:
[]

vgg_face:conv2_2, vgg material:conv2_2

unique for vgg face:
['white-c', 'blue-c']

unique for vgg:
['lacelike']

common concept:
['red-c']

vgg_face:conv3_3, vgg material:conv3_3

unique for vgg face:
[]

unique for vgg:
['zigzagged', 'cobwebbed', 'red-c', 'banded', 'hand', 'perforated', 'grid', 'lacelike', 'crosshatched', 'lined', 'woven']

common concept:
[]

vgg_face:conv4_3, vgg material:conv4_3

unique for vgg face:
[]

unique for vgg:
['meshed', 'swirly', 'porous', 'head', 'hair', 'bubbly', 'zigzagged', 'dotted', 'sprinkled', 'pitted', 'studded', 'polka-dotted', 'scaly', 'wheel', 'plant', 'ball', 'fibrous', 'hand', 'lacelike', 'ball_pit-s', 'striped', 'cracked', 'chequered', 'veined', 'cobwebbed', 'matted', 'paisley', 'grass', 'tree', 'dog', 'cat', 'person', 'waffled', 'perforated', 'honeycombed', 'lined', 'woven']

common concept:
['banded', 'grid']

vgg_face:conv5_3, vgg ma

In [12]:
# comparison between vgg face and vgg material
for blob in range(8):
    face = detect_concept_max(vgg_face, vgg_face["blobs"][blob])
    vgg_ = detect_concept_max(vgg_material ,vgg_material["blobs"][blob])
    print("vgg_face:{}, vgg material:{}\n".format(vgg_face["blobs"][blob], vgg_material["blobs"][blob]))
    print("unique for vgg face:\n{}\n".format(list(face - vgg_)))
    print("unique for vgg:\n{}\n".format(list(vgg_ - face)))
    print("common concept:\n{}\n".format(list(vgg_ & face)))

vgg_face:conv1_2, vgg material:conv1_2

unique for vgg face:
[]

unique for vgg:
[]

common concept:
['Unnamed: 1']

vgg_face:conv2_2, vgg material:conv2_2

unique for vgg face:
['oast_house-s']

unique for vgg:
['plant', 'cobwebbed', 'pink-c', 'swirly', 'banded', 'tree', 'paisley', 'orange-c', 'sprinkled', 'yellow-c', 'lacelike', 'studded', 'woven', 'green-c', 'ground']

common concept:
['bubbly', 'meshed', 'zigzagged', 'blue-c', 'red-c', 'white-c', 'perforated', 'grid', 'striped', 'lined', 'black-c', 'chequered']

vgg_face:conv3_3, vgg material:conv3_3

unique for vgg face:
['waffled']

unique for vgg:
['bicycle', 'tree', 'grooved', 'knitted', 'interlaced', 'bubbly', 'motorbike', 'sky', 'windowpane', 'yellow-c', 'skyscraper', 'fibrous', 'curtain', 'polka-dotted', 'scaly', 'wheel', 'plant', 'bus', 'hand', 'ball_pit-s', 'crosshatched', 'green-c', 'hair', 'cobwebbed', 'blue-c', 'porous', 'dog', 'person', 'wrinkled', 'cracked']

common concept:
['meshed', 'swirly', 'head', 'white-c', 'ar